# Algoritmos para Robótica Educativa

Algunos conceptos y algoritmos básicos que pueden aplicarse en competencias de robótica con Arduino, como resolución de laberintos y carreras sigue líneas.

## Resolución de laberintos

Un robot que resuelve laberintos puede usar métodos como el "algoritmo de la mano derecha/izquierda" o el "algoritmo de vuelta atrás (backtracking)".

### Algoritmo de la mano derecha/izquierda

Este es uno de los métodos más sencillos y consiste en seguir siempre el mismo lado del laberinto, girando a la derecha (o a la izquierda) en cada bifurcación. Este método funciona en laberintos "conexos", donde todos los caminos están conectados sin áreas aisladas.

**Pseudocódigo:**
1. Si hay una pared a la derecha, avanza.
2. Si el camino a la derecha está libre, gira a la derecha y avanza.
3. Si no puedes avanzar, gira a la izquierda hasta que encuentres un camino libre.

In [ ]:
#include <Servo.h>

Servo servoMotor;

void setup() {
  servoMotor.attach(9);  // Servo para girar
  pinMode(LEFT_SENSOR, INPUT);
  pinMode(RIGHT_SENSOR, INPUT);
  pinMode(FRONT_SENSOR, INPUT);
}

void loop() {
  if (digitalRead(RIGHT_SENSOR) == LOW) {
    // Gira a la derecha y avanza
    servoMotor.write(90); 
    delay(500); 
    avanza();
  } else if (digitalRead(FRONT_SENSOR) == LOW) {
    // Avanza
    avanza();
  } else if (digitalRead(LEFT_SENSOR) == LOW) {
    // Gira a la izquierda
    servoMotor.write(0);  
    delay(500);  
  } else {
    // Retrocede y gira
    retrocede();
  }
}

void avanza() {
  // Mueve los motores hacia adelante
}

void retrocede() {
  // Mueve los motores hacia atrás
}

### Optimización del algoritmo

Para mejorar el rendimiento, se puede utilizar una estrategia de mapa en el laberinto para evitar volver a caminos visitados. La **estrategia de mapa** en la resolución de laberintos implica que el robot cree un "mapa" o una "representación del entorno" mientras explora el laberinto. En lugar de simplemente seguir un patrón (como el método de la mano derecha/izquierda), el robot registra las rutas que ha explorado para optimizar su búsqueda y evitar pasar por caminos ya visitados. Esto puede hacer que el robot encuentre la salida más rápido, especialmente en laberintos más complejos o con caminos repetitivos.

#### Pasos de la Estrategia de Mapa

1. **Dividir el laberinto en celdas**: Imagina que el laberinto es una cuadrícula, cada intersección y cada giro se convierte en una celda o "nodo" del laberinto que el robot debe registrar.

2. **Registrar los movimientos**: Cada vez que el robot se mueve a una nueva celda o nodo, guarda su posición y las rutas disponibles. Esto se almacena en una estructura de datos, como una matriz o lista de nodos.

3. **Marcar caminos explorados**: Cada vez que el robot visita una celda, marca las rutas que ya ha probado y, en su siguiente movimiento, evita los caminos que ya ha explorado.

4. **Algoritmo de retroceso (Backtracking)**: Si el robot llega a un callejón sin salida o una celda donde ya exploró todas las rutas, utiliza un algoritmo de retroceso para regresar al último nodo no explorado y continuar desde allí.

5. **Detección de la salida**: A medida que el robot explora, revisa en cada nodo si ha alcanzado la salida. Si encuentra la salida, puede detenerse o registrar la ruta óptima.

In [ ]:
struct Nodo {
  int x, y;          // Posición en la cuadrícula
  bool arriba, abajo, izquierda, derecha;  // Rutas exploradas
  bool visitado;
};

Nodo laberinto[10][10]; // Supongamos que el laberinto es de 10x10

int x_actual = 0;
int y_actual = 0;

void setup() {
  // Inicialización de sensores y motores
  iniciarLaberinto();
}

void loop() {
  // Detecta caminos disponibles en la posición actual
  bool caminoArriba = !digitalRead(SENSOR_ARRIBA);
  bool caminoAbajo = !digitalRead(SENSOR_ABAJO);
  bool caminoIzquierda = !digitalRead(SENSOR_IZQUIERDA);
  bool caminoDerecha = !digitalRead(SENSOR_DERECHA);

  // Actualiza el nodo actual con las rutas disponibles
  actualizarNodo(x_actual, y_actual, caminoArriba, caminoAbajo, caminoIzquierda, caminoDerecha);

  // Busca una ruta no explorada y mueve el robot
  if (caminoArriba && !laberinto[x_actual][y_actual].arriba) {
    moverArriba();
  } else if (caminoDerecha && !laberinto[x_actual][y_actual].derecha) {
    moverDerecha();
  } else if (caminoAbajo && !laberinto[x_actual][y_actual].abajo) {
    moverAbajo();
  } else if (caminoIzquierda && !laberinto[x_actual][y_actual].izquierda) {
    moverIzquierda();
  } else {
    // Si todos los caminos ya fueron explorados, retrocede
    retroceder();
  }

  // Marca la posición actual como visitada
  laberinto[x_actual][y_actual].visitado = true;
}

// Función para iniciar el laberinto vacío
void iniciarLaberinto() {
  for (int i = 0; i < 10; i++) {
    for (int j = 0; j < 10; j++) {
      laberinto[i][j] = {i, j, false, false, false, false, false};
    }
  }
}

// Actualizar rutas exploradas del nodo
void actualizarNodo(int x, int y, bool arriba, bool abajo, bool izquierda, bool derecha) {
  laberinto[x][y].arriba = arriba;
  laberinto[x][y].abajo = abajo;
  laberinto[x][y].izquierda = izquierda;
  laberinto[x][y].derecha = derecha;
}

// Funciones para mover el robot en distintas direcciones
void moverArriba() { y_actual--; /* Código para mover */ }
void moverAbajo() { y_actual++; /* Código para mover */ }
void moverIzquierda() { x_actual--; /* Código para mover */ }
void moverDerecha() { x_actual++; /* Código para mover */ }
void retroceder() { /* Lógica de retroceso */ }


## Carrera sigue líneas

Para carreras sigue líneas, el robot sigue una línea en el suelo, generalmente negra sobre un fondo blanco, usando sensores de infrarrojos (IR). El algoritmo básico es un control proporcional (P) que corrige el movimiento según la desviación respecto a la línea.

### Algoritmo básico de sigue líneas
1. Usa sensores IR para detectar la línea.
2. Si el sensor izquierdo detecta la línea, gira ligeramente hacia la izquierda.
3. Si el sensor derecho detecta la línea, gira hacia la derecha.
4. Si ambos sensores están en la línea, avanza recto.
5. Si ambos sensores pierden la línea, retrocede hasta recuperarla.

In [ ]:
const int LEFT_SENSOR = A0;
const int RIGHT_SENSOR = A1;

void setup() {
  pinMode(LEFT_SENSOR, INPUT);
  pinMode(RIGHT_SENSOR, INPUT);
  pinMode(MOTOR_LEFT, OUTPUT);
  pinMode(MOTOR_RIGHT, OUTPUT);
}

void loop() {
  int leftValue = analogRead(LEFT_SENSOR);
  int rightValue = analogRead(RIGHT_SENSOR);

  if (leftValue < threshold && rightValue > threshold) {
    // Gira ligeramente a la izquierda
    setMotors(200, 100);
  } else if (rightValue < threshold && leftValue > threshold) {
    // Gira ligeramente a la derecha
    setMotors(100, 200);
  } else if (leftValue < threshold && rightValue < threshold) {
    // Avanza recto
    setMotors(200, 200);
  } else {
    // Retrocede hasta encontrar la línea
    setMotors(-100, -100);
  }
}

void setMotors(int leftSpeed, int rightSpeed) {
  analogWrite(MOTOR_LEFT, leftSpeed);
  analogWrite(MOTOR_RIGHT, rightSpeed);
}

### Optimización del algoritmo

Para mejorar el rendimiento, se puede ajustar las velocidades y agregar un controlador PID que permite ajustes más precisos y una mejor respuesta en curvas o trayectorias rápidas.

#### Esquema básico de implementación en Arduino:

1. **Hardware necesario:**
   - Sensores IR (al menos 3 para mejor precisión: izquierda, centro, derecha).
   - Motor de tracción (DC) con controlador como L298N.
   - Servo para dirección.

2. **Esquema lógico:**
   - Los sensores IR detectan la posición relativa de la línea (izquierda, centro, derecha).
   - Usar estas lecturas para calcular un "error" basado en la desviación respecto al centro.
   - Aplicar el control PID sobre este error para ajustar el servo y/o la velocidad del motor.

In [ ]:
#include <Servo.h>

// Definir pines
#define SENSOR_IZQ A0
#define SENSOR_CEN A1
#define SENSOR_DER A2
#define MOTOR_TRACCION 5
#define SERVO_DIR 9

// Variables para PID
float kp = 1.0, ki = 0.0, kd = 0.0;
float error = 0, prevError = 0, integral = 0, derivative = 0;

// Servo
Servo servo;

// Configuración inicial
void setup() {
  pinMode(SENSOR_IZQ, INPUT);
  pinMode(SENSOR_CEN, INPUT);
  pinMode(SENSOR_DER, INPUT);
  pinMode(MOTOR_TRACCION, OUTPUT);
  servo.attach(SERVO_DIR);
}

// Loop principal
void loop() {
  // Leer sensores
  int sensorIzq = digitalRead(SENSOR_IZQ);
  int sensorCen = digitalRead(SENSOR_CEN);
  int sensorDer = digitalRead(SENSOR_DER);

  // Calcular el error
  if (sensorCen == HIGH) error = 0;
  else if (sensorIzq == HIGH) error = -1;
  else if (sensorDer == HIGH) error = 1;

  // PID
  integral += error;
  derivative = error - prevError;
  float pid = (kp * error) + (ki * integral) + (kd * derivative);

  // Aplicar corrección al servo
  int servoPos = 90 + pid; // 90 es la posición central
  servo.write(constrain(servoPos, 0, 180));

  // Controlar velocidad del motor
  analogWrite(MOTOR_TRACCION, 150); // Velocidad constante (puedes ajustarla)

  // Actualizar error previo
  prevError = error;

  delay(10); // Pequeño delay para evitar ruido
}

#### Ajuste del PID:

- **Kp:** Incrementa la fuerza de corrección inicial. Si es muy alto, puede provocar oscilaciones.
- **Ki:** Corrige errores acumulados en el tiempo. Útil si el robot tiende a desviarse lentamente.
- **Kd:** Suaviza las oscilaciones y anticipa cambios.